In [1]:
import numpy as np
import pandas as pd



In [2]:
students=pd.read_csv("student-data.csv")

In [3]:
students.head()

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,internet,romantic,famrel,freetime,goout,Dalc,Walc,health,absences,passed
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,...,no,no,4,3,4,1,1,3,6,no
1,GP,F,17,U,GT3,T,1,1,at_home,other,...,yes,no,5,3,3,1,1,3,4,no
2,GP,F,15,U,LE3,T,1,1,at_home,other,...,yes,no,4,3,2,2,3,3,10,yes
3,GP,F,15,U,GT3,T,4,2,health,services,...,yes,yes,3,2,2,1,1,5,2,yes
4,GP,F,16,U,GT3,T,3,3,other,other,...,no,no,4,3,2,1,2,5,4,yes


##### find out the following facts about the dataset?
Total number of students?
Number of students who passed?
Number of students who failed?
Graduation rate of the class (%)?
Number of features?

In [15]:
totalstudents=students.shape[0]
totalpassed=students[students.passed=="yes"].shape[0]
totalfailed=students[students.passed=="no"].shape[0]
totalfeatures=len(students.columns)
gradpercent=round((totalpassed/float(totalstudents))*100,2)
print "Total number of students ",totalstudents
print "Total passed ",totalpassed
print "Total failed ",totalfailed
print "Total columns ",totalfeatures
print "Graduation percent ",gradpercent

Total number of students  395
Total passed  265
Total failed  130
Total columns  31
Graduation percent  67.09


In [19]:
target_cols=students.columns[-1]
feature_cols=students.columns[:-1]
X_all=students[feature_cols]
y_all=students[target_cols]
print "Feature Cols :",feature_cols
print "Target Cols :",target_cols

Feature Cols : Index([u'school', u'sex', u'age', u'address', u'famsize', u'Pstatus', u'Medu',
       u'Fedu', u'Mjob', u'Fjob', u'reason', u'guardian', u'traveltime',
       u'studytime', u'failures', u'schoolsup', u'famsup', u'paid',
       u'activities', u'nursery', u'higher', u'internet', u'romantic',
       u'famrel', u'freetime', u'goout', u'Dalc', u'Walc', u'health',
       u'absences'],
      dtype='object')
Target Cols : passed


In [23]:
#convert all the object columns to numeric
def preprocessing(X):
    outX=pd.DataFrame(index=X.index)
    for col,col_data in X.iteritems():
        if col_data.dtype==object:
            col_data=col_data.replace(["yes","no"],[1,0])
        if col_data.dtype==object:
            col_data=pd.get_dummies(col_data,prefix=col)
        outX=outX.join(col_data)
    return outX

X_all=preprocessing(X_all)
print list(X_all.columns)
print len(X_all.columns)

['school_GP', 'school_MS', 'sex_F', 'sex_M', 'age', 'address_R', 'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Medu', 'Fedu', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_father', 'guardian_mother', 'guardian_other', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']
48


In [25]:
#split the model into training and testing set
from sklearn.cross_validation import train_test_split
def shuffle_split_data(train_set_size):
    X_train,X_test,y_train,y_test=train_test_split(X_all,y_all,train_size=train_set_size,random_state=1)
    return X_train,X_test,y_train,y_test

X_train,X_test,y_train,y_test=shuffle_split_data(300)
print X_train.shape
print X_test.shape

(300, 48)
(95, 48)


In [29]:
#training and Evaluating Models
from sklearn.ensemble import RandomForestClassifier
import time

clf=RandomForestClassifier(n_estimators=10)
def train_classifier(clf,X_train,y_train):
    print "Model name",clf.__class__.__name__
    start=time.time()
    clf.fit(X_train,y_train)
    end=time.time()
    print "Done training in sec :{:.2f}".format(end-start)
    
train_classifier(clf,X_train,y_train)

Model name RandomForestClassifier
Done training in sec :0.05


In [34]:
from sklearn import metrics
def predict_accuracy(clf,features,target):
    print "Predicting for ",clf.__class__.__name__
    start=time.time()
    y_pred_class=clf.predict(features)
    end=time.time()
    print "Done!\nPrediction time (secs): {:.3f}".format(end - start)
    return metrics.f1_score(target.values, y_pred_class,pos_label='yes')

Trainingaccuracy=predict_accuracy(clf,X_train,y_train)
print "Training accuracy :",Trainingaccuracy
Testingaccuracy=predict_accuracy(clf,X_test,y_test)
print "Testing accuracy :",Testingaccuracy
    

Predicting for  RandomForestClassifier
Done!
Prediction time (secs): 0.010
Training accuracy : 0.992443324937
Predicting for  RandomForestClassifier
Done!
Prediction time (secs): 0.015
Testing accuracy : 0.828571428571


In [35]:
def train_predict(X_train,X_test,y_train,y_test):
    
    print "------------------------------------"
    print len(X_train)
    train_classifier(clf,X_train,y_train)
    print "Training accuracy {}".format(predict_accuracy(clf,X_train,y_train))
    print "Testing accuracy {}".format(predict_accuracy(clf,X_test,y_test))
    
X_train,X_test,y_train,y_test=shuffle_split_data(100)
train_predict(X_train,X_test,y_train,y_test)

X_train,X_test,y_train,y_test=shuffle_split_data(200)
train_predict(X_train,X_test,y_train,y_test)

X_train,X_test,y_train,y_test=shuffle_split_data(300)
train_predict(X_train,X_test,y_train,y_test)

------------------------------------
100
Model name RandomForestClassifier
Done training in sec :0.04
Predicting for  RandomForestClassifier
Done!
Prediction time (secs): 0.009
Training accuracy 1.0
Predicting for  RandomForestClassifier
Done!
Prediction time (secs): 0.007
Testing accuracy 0.740740740741
------------------------------------
200
Model name RandomForestClassifier
Done training in sec :0.03
Predicting for  RandomForestClassifier
Done!
Prediction time (secs): 0.010
Training accuracy 0.992424242424
Predicting for  RandomForestClassifier
Done!
Prediction time (secs): 0.008
Testing accuracy 0.721804511278
------------------------------------
300
Model name RandomForestClassifier
Done training in sec :0.03
Predicting for  RandomForestClassifier
Done!
Prediction time (secs): 0.005
Training accuracy 0.992405063291
Predicting for  RandomForestClassifier
Done!
Prediction time (secs): 0.005
Testing accuracy 0.77037037037


In [37]:
from sklearn.svm import SVC
clf=SVC()
X_train,X_test,y_train,y_test=shuffle_split_data(100)
train_predict(X_train,X_test,y_train,y_test)

X_train,X_test,y_train,y_test=shuffle_split_data(200)
train_predict(X_train,X_test,y_train,y_test)

X_train,X_test,y_train,y_test=shuffle_split_data(300)
train_predict(X_train,X_test,y_train,y_test)


------------------------------------
100
Model name SVC
Done training in sec :0.00
Predicting for  SVC
Done!
Prediction time (secs): 0.002
Training accuracy 0.860927152318
Predicting for  SVC
Done!
Prediction time (secs): 0.004
Testing accuracy 0.806722689076
------------------------------------
200
Model name SVC
Done training in sec :0.01
Predicting for  SVC
Done!
Prediction time (secs): 0.003
Training accuracy 0.860841423948
Predicting for  SVC
Done!
Prediction time (secs): 0.004
Testing accuracy 0.820512820513
------------------------------------
300
Model name SVC
Done training in sec :0.01
Predicting for  SVC
Done!
Prediction time (secs): 0.010
Training accuracy 0.858387799564
Predicting for  SVC
Done!
Prediction time (secs): 0.002
Testing accuracy 0.846153846154
